# Testing the non-Markovian Path Analisys Package

We are going to generate first a MC trajectory of a toy model 

In [37]:
import numpy as np

from NMpathAnalysis.nmtools.interval import Interval
from src.trajectory import *

In [38]:
'''
Function to mimic a MC simulation
'''
def mc_simulation(numsteps):
    x = 5
    I = Interval([0,100])
    mc_traj = []
    
    for i in range(numsteps):
        dx = random.uniform(-10,10)
        if (x + dx) in I:
            x = x + dx
        mc_traj.append(x)
    return np.array(mc_traj)

In [39]:
'''
Simple mapping funcion 
'''
def mapping_function(x):
    return int(x/10)

In [40]:
#Generating a short MC trajectoy
mc_trajectory = mc_simulation(10000)


## 1- Ensemble class (analysis of continuos trajectories)

Stores an esemble (array) of trajectories with extra functionalities. The ensemble could have a any number of trajectories including no trajectories at all.

### Transition probabilities

In [41]:
my_ensemble = Ensemble(mc_trajectory)

In [42]:
n_states = 10
C1 = my_ensemble._count_matrix(n_states, mapping_function)
print(C1)

[[ 573.  199.    0.    0.    0.    0.    0.    0.    0.    0.]
 [ 199.  403.  234.    0.    0.    0.    0.    0.    0.    0.]
 [   0.  234.  462.  206.    0.    0.    0.    0.    0.    0.]
 [   0.    0.  206.  486.  241.    0.    0.    0.    0.    0.]
 [   0.    0.    0.  241.  455.  260.    0.    0.    0.    0.]
 [   0.    0.    0.    0.  260.  605.  293.    0.    0.    0.]
 [   0.    0.    0.    0.    0.  293.  509.  297.    0.    0.]
 [   0.    0.    0.    0.    0.    0.  297.  605.  295.    0.]
 [   0.    0.    0.    0.    0.    0.    0.  295.  546.  261.]
 [   0.    0.    0.    0.    0.    0.    0.    0.  261.  783.]]


In [43]:
K1 = my_ensemble._mle_transition_matrix(n_states, mapping_function)
print(K1)

[[ 0.74222798  0.25777202  0.          0.          0.          0.          0.
   0.          0.          0.        ]
 [ 0.23803828  0.48205742  0.27990431  0.          0.          0.          0.
   0.          0.          0.        ]
 [ 0.          0.2594235   0.51219512  0.22838137  0.          0.          0.
   0.          0.          0.        ]
 [ 0.          0.          0.22079314  0.52090032  0.25830654  0.          0.
   0.          0.          0.        ]
 [ 0.          0.          0.          0.25209205  0.47594142  0.27196653
   0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.22452504  0.5224525
   0.25302245  0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.26660601
   0.46314832  0.27024568  0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.2481203   0.50543024  0.24644946  0.        ]
 [ 0.          0.          0.          0.

### Defining states and computing MFPTs
The states are considered intervals in the is the class is Ensemble

In [44]:
stateA = [0,10]
stateB = [90,100]

my_ensemble.mfpts(stateA, stateB)

{'mfptAB': 198.33333333333334,
 'mfptBA': 271.6190476190476,
 'std_err_mfptAB': 21.137636993212855,
 'std_err_mfptBA': 41.809386928427422}

### Adding more trajectories to the ensemble or (ensemble + ensemble)

In [45]:
seq1 = mc_simulation(5000)
seq2 = mc_simulation(5000)
seq3 = mc_simulation(5000)

my_e1 = Ensemble(seq1)
my_e2 = Ensemble(seq2)
my_e3 = Ensemble(seq3)

ensemble1 = my_e1 + my_e2 + my_e3

In [46]:
stateA = [0,10]
stateB = [90,100]
ensemble1.mfpts(stateA,stateB)

{'mfptAB': 317.1666666666667,
 'mfptBA': 312.95454545454544,
 'std_err_mfptAB': 56.010095402856059,
 'std_err_mfptBA': 53.147365579764347}

In [47]:
e1 = Ensemble([1,2,3,4])
e2 = Ensemble([2,3,4,5])
e3 = Ensemble([2,1,1,4])

my_ensembles = [e1, e2, e3]

ensemble_tot = Ensemble([])

for traj in my_ensembles:
    ensemble_tot += traj

ensemble_tot.sequence
ensemble_tot.mfpts([1,1],[4,4])

{'mfptAB': 2.5,
 'mfptBA': 'NaN',
 'std_err_mfptAB': 0.35355339059327373,
 'std_err_mfptBA': 'NaN'}


## 2- DiscreteEnsemble class

We can generate a discrete trajectory from the same mapping function and we should obtain exaclty the same result:

In [48]:

d_ens = DiscreteEnsemble.from_ensemble(mc_trajectory,mapping_function)

print(d_ens.sequence[0][1:200])
d_ens.sequence.shape

[1 1 1 1 2 2 1 2 2 2 2 2 3 2 2 2 1 1 1 1 1 2 2 2 1 1 1 0 1 1 2 3 3 3 3 3 3
 2 2 2 2 1 2 1 0 0 0 1 0 0 1 1 2 1 1 1 0 1 1 2 2 3 3 4 4 5 5 6 7 7 7 6 5 5
 5 6 5 5 5 5 5 4 5 5 5 5 5 4 3 4 3 4 3 3 3 3 2 2 2 1 1 2 2 2 2 3 3 3 3 4 3
 4 4 4 4 3 3 3 4 5 6 6 6 7 7 6 6 6 6 5 6 5 6 7 7 7 7 7 7 6 7 8 7 6 7 7 8 8
 8 8 8 9 8 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 8 8 9 9 9 8 8 8 8 8 9 9 9
 9 9 9 9 9 9 8 8 8 9 8 8 7 6]


(1, 10000)

In [49]:
C2 = d_ens._count_matrix(n_states)
print(C2)

[[ 573.  199.    0.    0.    0.    0.    0.    0.    0.    0.]
 [ 199.  403.  234.    0.    0.    0.    0.    0.    0.    0.]
 [   0.  234.  462.  206.    0.    0.    0.    0.    0.    0.]
 [   0.    0.  206.  486.  241.    0.    0.    0.    0.    0.]
 [   0.    0.    0.  241.  455.  260.    0.    0.    0.    0.]
 [   0.    0.    0.    0.  260.  605.  293.    0.    0.    0.]
 [   0.    0.    0.    0.    0.  293.  509.  297.    0.    0.]
 [   0.    0.    0.    0.    0.    0.  297.  605.  295.    0.]
 [   0.    0.    0.    0.    0.    0.    0.  295.  546.  261.]
 [   0.    0.    0.    0.    0.    0.    0.    0.  261.  783.]]


In [50]:
K2= d_ens._mle_transition_matrix(n_states)
print(K2)

[[ 0.74222798  0.25777202  0.          0.          0.          0.          0.
   0.          0.          0.        ]
 [ 0.23803828  0.48205742  0.27990431  0.          0.          0.          0.
   0.          0.          0.        ]
 [ 0.          0.2594235   0.51219512  0.22838137  0.          0.          0.
   0.          0.          0.        ]
 [ 0.          0.          0.22079314  0.52090032  0.25830654  0.          0.
   0.          0.          0.        ]
 [ 0.          0.          0.          0.25209205  0.47594142  0.27196653
   0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.22452504  0.5224525
   0.25302245  0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.26660601
   0.46314832  0.27024568  0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.2481203   0.50543024  0.24644946  0.        ]
 [ 0.          0.          0.          0.

### Defining states and computing MFPTs
The states are now considered sets, defining the states as follow we should obtain the same results

In [51]:
stateA = [0]
stateB = [9]

d_ens.mfpts(stateA, stateB)

{'mfptAB': 198.33333333333334,
 'mfptBA': 271.6190476190476,
 'std_err_mfptAB': 21.137636993212855,
 'std_err_mfptBA': 41.809386928427422}

The main function of this class would be to generate trajectories from the transition matrix

In [56]:
d_ens2 = DiscreteEnsemble.from_transition_matrix(K2, sim_length = 10000)

In [57]:
d_ens2.mfpts(stateA,stateB)

{'mfptAB': 141.85714285714286,
 'mfptBA': 333.5238095238095,
 'std_err_mfptAB': 21.14653396968404,
 'std_err_mfptBA': 69.294848654878962}